## Section 1: Dependencies and Data Loading

In [ ]:
import sys
sys.path.insert(0, '/Users/ajaiupadhyaya/Documents/Models')

from models.ml import LSTMPredictor
from core.backtesting import BacktestEngine

import pandas as pd
import numpy as np
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded.")
print("\nNote: LSTM requires TensorFlow:")
print("  pip install tensorflow")

# Check TensorFlow availability
try:
    import tensorflow as tf
    print(f"\n✓ TensorFlow {tf.__version__} available")
    TF_AVAILABLE = True
except ImportError:
    print("\n⚠ TensorFlow not available (install for LSTM training)")
    TF_AVAILABLE = False

## Section 2: Download and Prepare Market Data

In [ ]:
# Download data
print("Downloading market data for LSTM training...")
tickers = ['SPY', 'QQQ', 'IWM']  # Different assets for comparison
dfs = {}

for ticker in tickers:
    df = yf.download(ticker, period='3y', progress=False)
    dfs[ticker] = df
    print(f"{ticker}: {len(df)} days, price range ${df['Close'].min():.2f} - ${df['Close'].max():.2f}")

# Focus on SPY for primary training
df = dfs['SPY']
print(f"\nUsing SPY for LSTM training ({len(df)} days)")

## Section 3: Data Preprocessing for LSTM

In [ ]:
# Calculate returns and indicators
df['returns'] = df['Close'].pct_change()
df['SMA_5'] = df['Close'].rolling(window=5).mean()
df['SMA_20'] = df['Close'].rolling(window=20).mean()
df['volatility'] = df['returns'].rolling(window=20).std()

# Remove NaN
df = df.dropna()

print("Data preprocessing:")
print(f"  Features: Close, High, Low, Volume, Returns, SMA_5, SMA_20, Volatility")
print(f"  Data points: {len(df)}")
print(f"\nData statistics:")
print(f"  Close: ${df['Close'].mean():.2f} ± ${df['Close'].std():.2f}")
print(f"  Returns: {df['returns'].mean()*100:.3f}% ± {df['returns'].std()*100:.2f}%")
print(f"  Volume: {df['Volume'].mean()/1e6:.1f}M ± {df['Volume'].std()/1e6:.1f}M")

## Section 4: Train LSTM Model on SPY

In [ ]:
if TF_AVAILABLE:
    print("Initializing LSTM model...\n")
    
    lstm = LSTMPredictor(lookback_window=20)
    
    print(f"LSTM Configuration:")
    print(f"  Lookback window: 20 periods")
    print(f"  Architecture: 2 LSTM layers (64 units) + Dropout")
    print(f"  Training data: {len(df)} days\n")
    
    # Prepare data
    print("Preparing sequences...")
    lstm.prepare_data(df)
    print(f"  X shape: {lstm.X.shape}")
    print(f"  y shape: {lstm.y.shape}\n")
    
    # Build model
    print("Building LSTM model...")
    lstm.build_model()
    print("Model built successfully!\n")
    
    # Train
    print("Training LSTM (100 epochs)...")
    history = lstm.train(epochs=100, batch_size=32, verbose=0)
    print("Training complete!\n")
    
    # Show training history
    print("Training Results:")
    print(f"  Initial loss: {history.history['loss'][0]:.6f}")
    print(f"  Final loss: {history.history['loss'][-1]:.6f}")
    print(f"  Loss reduction: {(1 - history.history['loss'][-1]/history.history['loss'][0])*100:.1f}%")

else:
    print("LSTM training requires TensorFlow installation.")
    print("Install with: pip install tensorflow")

## Section 5: Generate Predictions and Backtest

In [ ]:
if TF_AVAILABLE:
    print("Generating LSTM predictions...\n")
    
    # Predict on same data (for demonstration - use walk-forward for real trading)
    lstm_signals = lstm.predict(df)
    
    print(f"Signal Statistics:")
    print(f"  Mean: {np.mean(lstm_signals):.4f}")
    print(f"  Std: {np.std(lstm_signals):.4f}")
    print(f"  Min: {np.min(lstm_signals):.4f}")
    print(f"  Max: {np.max(lstm_signals):.4f}")
    print(f"  Buy signals (>0.3): {sum(lstm_signals > 0.3)}")
    print(f"  Sell signals (<-0.3): {sum(lstm_signals < -0.3)}\n")
    
    # Backtest LSTM signals
    print("Backtesting LSTM strategy...")
    engine = BacktestEngine(initial_capital=100000, commission=0.001)
    results = engine.run_backtest(df, lstm_signals, signal_threshold=0.3, position_size=0.1)
    
    print(f"\nLSTM Trading Results:")
    print(f"  Final Equity: ${results['final_equity']:,.2f}")
    print(f"  Total Return: {results['total_return_pct']:.2f}%")
    print(f"  Total Trades: {results['num_trades']}")
    print(f"  Win Rate: {results['win_rate']*100:.1f}%")
    print(f"  Sharpe Ratio: {results['sharpe_ratio']:.2f}")
    print(f"  Max Drawdown: {results['max_drawdown_pct']:.2f}%")
else:
    print("Prediction requires TensorFlow.")

## Section 6: Multi-Asset Training

In [ ]:
if TF_AVAILABLE:
    print("Training LSTM on multiple assets...\n")
    
    results_by_asset = {}
    
    for ticker in tickers:
        print(f"Training LSTM on {ticker}...")
        
        df_asset = dfs[ticker].copy()
        df_asset['returns'] = df_asset['Close'].pct_change()
        df_asset['SMA_5'] = df_asset['Close'].rolling(5).mean()
        df_asset['SMA_20'] = df_asset['Close'].rolling(20).mean()
        df_asset['volatility'] = df_asset['returns'].rolling(20).std()
        df_asset = df_asset.dropna()
        
        # Train LSTM
        lstm_asset = LSTMPredictor(lookback_window=20)
        lstm_asset.prepare_data(df_asset)
        lstm_asset.build_model()
        lstm_asset.train(epochs=50, batch_size=32, verbose=0)
        
        # Generate signals and backtest
        signals = lstm_asset.predict(df_asset)
        engine = BacktestEngine(initial_capital=100000, commission=0.001)
        result = engine.run_backtest(df_asset, signals, signal_threshold=0.3, position_size=0.1)
        
        results_by_asset[ticker] = result
        print(f"  Return: {result['total_return_pct']:.2f}%, Sharpe: {result['sharpe_ratio']:.2f}\n")
    
    # Compare across assets
    print("LSTM Performance Across Assets:")
    print(f"\n{'Asset':<10} {'Return':<12} {'Win Rate':<12} {'Sharpe':<10}")
    print("-" * 44)
    
    for ticker, result in results_by_asset.items():
        print(f"{ticker:<10} {result['total_return_pct']:>6.2f}% {result['win_rate']*100:>10.1f}% {result['sharpe_ratio']:>9.2f}")

else:
    print("Multi-asset training requires TensorFlow.")

## Section 7: Ensemble with Ensemble ML Model

In [ ]:
if TF_AVAILABLE:
    print("Creating ensemble of LSTM + Gradient Boosting...\n")
    
    from models.ml import EnsemblePredictor
    
    # Reset to SPY
    df_spy = dfs['SPY'].copy()
    df_spy['returns'] = df_spy['Close'].pct_change()
    df_spy['SMA_5'] = df_spy['Close'].rolling(5).mean()
    df_spy['SMA_20'] = df_spy['Close'].rolling(20).mean()
    df_spy['volatility'] = df_spy['returns'].rolling(20).std()
    df_spy = df_spy.dropna()
    
    # Split data
    split = int(len(df_spy) * 0.7)
    df_train = df_spy.iloc[:split]
    df_test = df_spy.iloc[split:]
    
    # Train both models
    print("Training LSTM on training set...")
    lstm = LSTMPredictor(lookback_window=20)
    lstm.prepare_data(df_train)
    lstm.build_model()
    lstm.train(epochs=50, batch_size=32, verbose=0)
    
    print("Training Ensemble on training set...")
    ensemble = EnsemblePredictor(lookback_window=20)
    ensemble.train(df_train)
    
    # Predict on test set
    lstm_test_signals = lstm.predict(df_test)
    ensemble_test_signals = ensemble.predict(df_test)
    
    # Combine signals (average)
    combined_signals = (lstm_test_signals + ensemble_test_signals) / 2
    
    # Backtest all three
    engine = BacktestEngine(initial_capital=100000, commission=0.001)
    
    lstm_result = engine.run_backtest(df_test, lstm_test_signals, signal_threshold=0.3, position_size=0.1)
    ensemble_result = engine.run_backtest(df_test, ensemble_test_signals, signal_threshold=0.3, position_size=0.1)
    combined_result = engine.run_backtest(df_test, combined_signals, signal_threshold=0.3, position_size=0.1)
    
    print(f"\nEnsemble Comparison (Test Set):")
    print(f"\n{'Model':<20} {'Return':<12} {'Sharpe':<10} {'Max DD':<10}")
    print("-" * 52)
    print(f"{'LSTM':<20} {lstm_result['total_return_pct']:>6.2f}% {lstm_result['sharpe_ratio']:>9.2f} {lstm_result['max_drawdown_pct']:>8.2f}%")
    print(f"{'Ensemble ML':<20} {ensemble_result['total_return_pct']:>6.2f}% {ensemble_result['sharpe_ratio']:>9.2f} {ensemble_result['max_drawdown_pct']:>8.2f}%")
    print(f"{'Combined (50/50)':<20} {combined_result['total_return_pct']:>6.2f}% {combined_result['sharpe_ratio']:>9.2f} {combined_result['max_drawdown_pct']:>8.2f}%")
else:
    print("Ensemble requires TensorFlow.")

## Section 8: Neural Network Architecture

In [ ]:
print("\nLSTM Architecture Summary:\n")
print("Layer 1: Input (20 timesteps, 5 features)")
print("Layer 2: LSTM (64 units, return_sequences=True)")
print("Layer 3: Dropout (20%)")
print("Layer 4: LSTM (64 units)")
print("Layer 5: Dropout (20%)")
print("Layer 6: Dense (1 unit, tanh activation) → signal (-1 to 1)")
print("\nOptimizer: Adam (learning_rate=0.001)")
print("Loss: Mean Squared Error")
print("Validation Split: 20%")
print(f"\nTotal Parameters: ~{5*20*64 + 64*64 + 64*64 + 64*1:,}")
print("\nWhy LSTM for trading?")
print("  - Captures temporal dependencies in price sequences")
print("  - Long-term memory: remembers important patterns over weeks")
print("  - Dropout prevents overfitting to historical noise")
print("  - Tanh output scales to [-1, 1] for trading signals")

## Section 9: Advanced Topics

print("\nAdvanced LSTM Enhancements:\n")
print("1. ATTENTION MECHANISMS")
print("   - Focus on important timesteps")
print("   - Interpretable: see which past days influenced prediction")
print("\n2. BIDIRECTIONAL LSTM")
print("   - Process sequences forward AND backward")
print("   - Better capture of context from both directions")
print("\n3. MULTI-TASK LEARNING")
print("   - Predict price direction + magnitude simultaneously")
print("   - Shared hidden layers improve generalization")
print("\n4. VARIATIONAL AUTOENCODERS")
print("   - Learn efficient price representations")
print("   - Detect regime changes and anomalies")
print("\n5. TRANSFORMER MODELS")
print("   - Self-attention for parallel computation")
print("   - Faster training on long sequences")
print("   - State-of-the-art performance on time series")
print("\n6. HYBRID MODELS")
print("   - LSTM + GRU + Attention + Ensemble")
print("   - Weighted combination based on market regime")

## Section 10: Production Deployment

print("="*70)
print("DEEP LEARNING TRADING PIPELINE")
print("="*70)

print(f"\n✓ COMPLETED:")
print(f"  1. LSTM model with TensorFlow/Keras")
print(f"  2. Sequence preparation (20-period lookback)")
print(f"  3. Multi-asset training and evaluation")
print(f"  4. Backtesting with signal generation")
print(f"  5. Ensemble with Gradient Boosting")
print(f"  6. Architecture explanation and rationale")

print(f"\n🚀 DEPLOYMENT READY:")
print(f"  - Export model to SavedModel format")
print(f"  - TensorFlow Serving for high-throughput inference")
print(f"  - Mobile/Edge deployment via TFLite")
print(f"  - Real-time prediction API (FastAPI)")
print(f"  - Model versioning and A/B testing")
print(f"  - Automated retraining pipeline")

print(f"\n📊 MONITORING & MAINTENANCE:")
print(f"  - Performance tracking (Sharpe, Sortino, Calmar ratios)")
print(f"  - Data drift detection")
print(f"  - Model degradation alerts")
print(f"  - A/B testing new models against live production")
print(f"  - Trade attribution and analysis")

print(f"\n" + "="*70)